<a href="https://colab.research.google.com/github/EngineeringErgonomics/AnkiBrain/blob/main/Predicting_Stock_Prices_with_LSTM_Neural_Networks_1_3-tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-ta

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pandas_ta as ta  # noqa
import matplotlib.pyplot as plt

# Step 1: Prepare the data
def create_sequences(feature_data, target_data, sequence_length):
    X, y = [], []
    for i in range(len(feature_data) - sequence_length):
        X.append(feature_data[i:(i + sequence_length)])
        y.append(target_data[i + sequence_length])
    return np.array(X), np.array(y)

# Define the sequence length (e.g., 30 days of historical data)
sequence_length = 90

# Select features for the model
features = ['Open', 'High', 'Low', 'Volume', 'close_delta', 'open_delta', 'high_delta', 'low_delta', 'volume_delta',
            'ema_8', 'ema_21', 'ema_34', 'ema_55', 'ema_89', 'macd_line', 'macd_signal_line', 'macd_histogram',
            'squeeze_pro_indicator', 'squeeze_pro_on_wide', 'squeeze_pro_on_normal', 'squeeze_pro_on_narrow',
            'squeeze_pro_off', 'squeeze_pro_no', 'squeeze_pro_increase', 'squeeze_pro_decrease',
            'squeeze_pro_potential_increase', 'squeeze_pro_potential_decrease', 'squeeze_pro_negative_decrease',
            'squeeze_pro_negative_increase']

target = 'Close'


In [3]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
# import tensorflow as tf
# print(tf.__version__)

# # Detect and init the TPU
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#     print('TPU detected')
#     print(f'TPU: {tpu}')
#     print(f'TPU Core Count: {tpu.num_accelerators()}')
#     print(f'TPU Cluster Spec: {tpu.cluster_spec()}')

#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     tpu_strategy = tf.distribute.TPUStrategy(tpu)
#     print("TPU Strategy created successfully")
#     print(f"Number of replicas in sync: {tpu_strategy.num_replicas_in_sync}")
# except ValueError as e:
#     print("Error while setting up TPU strategy:")
#     print(e)
#     tpu = None
#     tpu_strategy = None

# if tpu_strategy:
#     print("TPU is available and initialized")
# else:
#     print("TPU is not available. Using default strategy")
#     strategy = tf.distribute.get_strategy()
#     print(f"Current strategy: {strategy}")

In [5]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [6]:
from google.colab import drive
drive.mount("/content/drive")
content_root: str = "/content/drive/MyDrive/0SYSTEM/Masters_Program/data"
df: pd.DataFrame = pd.read_csv(f'{content_root}/SPY.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df["close_delta"]: pd.DataFrame = df.Close.diff().fillna(0)
df["open_delta"]: pd.DataFrame = df.Open.diff().fillna(0)
df["high_delta"]: pd.DataFrame = df.High.diff().fillna(0)
df["low_delta"]: pd.DataFrame = df.Low.diff().fillna(0)
df["volume_delta"]: pd.DataFrame = df.Volume.diff().fillna(0)
try:
    df.drop(["Adj Close", "Date"], axis=1, inplace=True)
except KeyError:
    pass

df

,Open,High,Low,Close,Volume,close_delta,open_delta,high_delta,low_delta,volume_delta
0,43.968750,43.968750,43.750000,43.937500,1003200,0.000000,0.000000,0.000000,0.000000,0.0
1,43.968750,44.250000,43.968750,44.250000,480500,0.312500,0.000000,0.281250,0.218750,-522700.0
2,44.218750,44.375000,44.125000,44.343750,201300,0.093750,0.250000,0.125000,0.156250,-279200.0
3,44.406250,44.843750,44.375000,44.812500,529400,0.468750,0.187500,0.468750,0.250000,328100.0
4,44.968750,45.093750,44.468750,45.000000,531500,0.187500,0.562500,0.250000,0.093750,2100.0
...,...,...,...,...,...,...,...,...,...,...
7949,563.179993,563.909973,559.049988,560.789978,35788600,-1.340027,3.649964,0.819946,1.760010,-14850800.0
7950,559.489990,562.059998,558.320007,561.559998,32693900,0.770020,-3.690003,-1.849975,-0.729981,-3094700.0
7951,561.210022,561.650024,555.039978,558.299988,41066000,-3.260010,1.720032,-0.409974,-3.280029,8372100.0
7952,560.309998,563.679993,557.179993,558.349976,38715200,0.049988,-0.900024,2.029969,2.140015,-2350800.0


In [8]:
ticker_data = df

In [9]:
# Calculate EMAs
ticker_data["ema_8"] = ticker_data["Close"].ewm(span=8, adjust=False).mean()
ticker_data["ema_21"] = ticker_data["Close"].ewm(span=21, adjust=False).mean()
ticker_data["ema_34"] = ticker_data["Close"].ewm(span=34, adjust=False).mean()
ticker_data["ema_55"] = ticker_data["Close"].ewm(span=55, adjust=False).mean()
ticker_data["ema_89"] = ticker_data["Close"].ewm(span=89, adjust=False).mean()

In [10]:
# Calculate MACD
try:
    macd = ticker_data.ta.macd(
        close="Close", fast=12, slow=26, signal=9, append=True
    )
    ticker_data["macd_line"] = macd["MACD_12_26_9"]
    ticker_data["macd_signal_line"] = macd["MACDs_12_26_9"]
    ticker_data["macd_histogram"] = macd["MACDh_12_26_9"]
except TypeError:
    # Handle missing or None values
    ticker_data["macd_line"] = None
    ticker_data["macd_signal_line"] = None
    ticker_data["macd_histogram"] = None

In [11]:
# Calculate Squeeze Pro Indicators
squeeze = ticker_data.ta.squeeze_pro(lazybear=True, detailed=True, append=True)
ticker_data["squeeze_pro_indicator"] = squeeze["SQZPRO_20_2.0_20_2_1.5_1"]
ticker_data["squeeze_pro_on_wide"] = squeeze["SQZPRO_ON_WIDE"]
ticker_data["squeeze_pro_on_normal"] = squeeze["SQZPRO_ON_NORMAL"]
ticker_data["squeeze_pro_on_narrow"] = squeeze["SQZPRO_ON_NARROW"]
ticker_data["squeeze_pro_off"] = squeeze["SQZPRO_OFF"]
ticker_data["squeeze_pro_no"] = squeeze["SQZPRO_NO"]
ticker_data["squeeze_pro_increase"] = squeeze["SQZPRO_INC"]
ticker_data["squeeze_pro_decrease"] = squeeze["SQZPRO_DEC"]
ticker_data["squeeze_pro_potential_increase"] = squeeze["SQZPRO_PINC"]
ticker_data["squeeze_pro_potential_decrease"] = squeeze["SQZPRO_PDEC"]
ticker_data["squeeze_pro_negative_decrease"] = squeeze["SQZPRO_NDEC"]
ticker_data["squeeze_pro_negative_increase"] = squeeze["SQZPRO_NINC"]

In [12]:
df = ticker_data

In [13]:
df_filled = df.ffill()
# Now apply backward fill
df_filled = df_filled.bfill()
df = df_filled

In [14]:
print(df.describe())
print("\nNull values:")
print(df.isnull().sum())
print("\nInfinite values:")
print(np.isinf(df).sum())

              Open         High          Low        Close        Volume  \
count  7954.000000  7954.000000  7954.000000  7954.000000  7.954000e+03   
mean    177.849266   178.899106   176.697239   177.860691  8.401158e+07   
std     116.974219   117.565903   116.347727   117.005731  9.145994e+07   
min      43.343750    43.531250    42.812500    43.406250  5.200000e+03   
25%     106.412503   107.285000   105.500000   106.381250  1.056615e+07   
50%     133.154999   133.950004   132.245002   133.194999  6.289980e+07   
75%     216.992500   217.352497   216.020001   216.935001  1.140842e+08   
max     563.179993   565.159973   562.099976   564.859985  8.710263e+08   

       close_delta   open_delta   high_delta    low_delta  volume_delta  ...  \
count  7954.000000  7954.000000  7954.000000  7954.000000  7.954000e+03  ...   
mean      0.065344     0.064974     0.065405     0.064545  7.752653e+03  ...   
std       2.318545     2.277337     1.876077     2.129497  3.790915e+07  ...   
min 

In [16]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
import numpy as np

# First, split the data
train_size = int(0.6 * len(df))
val_size = int(0.2 * len(df))

train_df = df[:train_size]
val_df = df[train_size:train_size+val_size]
test_df = df[train_size+val_size:]

# Initialize a single scaler for all features and target
scaler = StandardScaler()

def prepare_data(data, is_training=False):
    global scaler
    feature_data = data[features].values
    target_data = data[target].values.reshape(-1, 1)

    # Combine features and target for scaling
    combined_data = np.hstack((feature_data, target_data))

    if is_training:
        scaled_data = scaler.fit_transform(combined_data)
    else:
        scaled_data = scaler.transform(combined_data)

    # Split the scaled data back into features and target
    scaled_feature_data = scaled_data[:, :-1]
    scaled_target_data = scaled_data[:, -1]

    return scaled_feature_data, scaled_target_data

# Prepare the datasets
X_train, y_train = prepare_data(train_df, is_training=True)
X_val, y_val = prepare_data(val_df)
X_test, y_test = prepare_data(test_df)

# Create sequences
def create_sequences(feature_data, target_data, sequence_length):
    X, y = [], []
    for i in range(len(feature_data) - sequence_length):
        X.append(feature_data[i:(i + sequence_length)])
        y.append(target_data[i + sequence_length])
    return np.array(X), np.array(y)

# Create sequences for each dataset
sequence_length = 30  # Make sure this is defined
X_train, y_train = create_sequences(X_train, y_train, sequence_length)
X_val, y_val = create_sequences(X_val, y_val, sequence_length)
X_test, y_test = create_sequences(X_test, y_test, sequence_length)

print("Shapes after sequence creation:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# Build the LSTM model
input_shape = (sequence_length, len(features))
first_layer = len(features)
second_layer = int(first_layer ** 2)
third_layer = int(second_layer * 2)
fourth_layer = int(third_layer / 2)
fifth_layer = int(fourth_layer / 2)
sixth_layer = int(fifth_layer / 2)
with tpu_strategy.scope():
    inputs = tf.keras.layers.Input(shape=(sequence_length, len(features)))

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(first_layer, activation='relu', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01)))(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(second_layer, activation='relu', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01)))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(third_layer, activation='relu', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01)))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(fourth_layer, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    x = tf.keras.layers.Dense(fifth_layer, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

# Print model summary
model.summary()

Shapes after sequence creation:
X_train: (4742, 30, 29), y_train: (4742,)
X_val: (1560, 30, 29), y_val: (1560,)
X_test: (1562, 30, 29), y_test: (1562,)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 30, 29)]          0         
                                                                 
 bidirectional (Bidirection  (None, 30, 58)            13688     
 al)                                                             
                                                                 
 batch_normalization (Batch  (None, 30, 58)            232       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 30, 58)            0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 30, 1

In [ ]:

# Step 3: Train the model
from tensorflow.keras import callbacks
from tensorflow.keras.models import load_model
from datetime import datetime

# get current year-month-day-hour_minute
current_datetime = datetime.now().strftime("%Y-%m-%d-%H_%M")
model_name = f"{content_root}/best_models/model_{current_datetime}.keras"

# Define callbacks
checkpoint = callbacks.ModelCheckpoint(model_name, save_best_only=True, monitor='val_loss', mode='min', verbose=1)
early_stopping = callbacks.EarlyStopping(patience=15, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)

batch_size = 32 * tpu_strategy.num_replicas_in_sync  # Adjust batch size for TPU

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size, drop_remainder=True)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size, drop_remainder=True)

with tpu_strategy.scope():
    history = model.fit(
        train_dataset,
        epochs=100,
        validation_data=val_dataset,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
    )

Epoch 1/100
18/18 [==============================] - 35s 533ms/step - loss: 138.0279 - val_loss: 172919742918156317884416.0000
Epoch 2/100
18/18 [==============================] - 6s 350ms/step - loss: 143.9591 - val_loss: 280791.2500
Epoch 3/100
18/18 [==============================] - 6s 346ms/step - loss: 146.0774 - val_loss: 612.7119
Epoch 4/100
18/18 [==============================] - 7s 386ms/step - loss: 145.7959 - val_loss: 282.0374
Epoch 5/100
18/18 [==============================] - 6s 352ms/step - loss: 145.0168 - val_loss: 202.5404
Epoch 6/100
18/18 [==============================] - 6s 353ms/step - loss: 144.2707 - val_loss: 152.4530
Epoch 7/100
18/18 [==============================] - 6s 353ms/step - loss: 143.6134 - val_loss: 152.2061
Epoch 8/100
18/18 [==============================] - 6s 353ms/step - loss: 142.9663 - val_loss: 149.5323
Epoch 9/100
18/18 [==============================] - 6s 325ms/step - loss: 142.4254 - val_loss: 154.3381
Epoch 10/100
18/18 [==========

In [ ]:
best_model = load_model(model_name)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size, drop_remainder=True)

with tpu_strategy.scope():
    test_loss = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")

In [ ]:

# Step 4: Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def evaluate_model(model, X, y_true, scaler):
    y_pred = model.predict(X)
    y_pred_original = scaler.inverse_transform(y_pred)
    y_true_original = scaler.inverse_transform(y_true.reshape(-1, 1))

    mse = mean_squared_error(y_true_original, y_pred_original)
    mae = mean_absolute_error(y_true_original, y_pred_original)
    mape = mean_absolute_percentage_error(y_true_original, y_pred_original)

    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.4f}")

# Evaluate on each set
print("Train set evaluation:")
evaluate_model(model, X_train, y_train, target_scaler)
print("\nValidation set evaluation:")
evaluate_model(model, X_val, y_val, target_scaler)
print("\nTest set evaluation:")
evaluate_model(model, X_test, y_test, target_scaler)


In [ ]:
# Step 5: Make predictions
with tpu_strategy.scope():
    predictions = model.predict(test_dataset)
# Get the last sequence from X_train
last_train_sequence = X_train[-1]

In [ ]:


# Inverse transform predictions
predicted_closes = target_scaler.inverse_transform(predictions).flatten()

# Prepare dummy array for actual values
dummy_actual = np.zeros((y_test.shape[0], len(features)))
dummy_actual[:] = last_train_sequence[-1]  # Fill with the last time step of the last training sequence
dummy_actual[:, -1] = y_test  # Replace the last column with actual values

# Inverse transform actual values
actual_closes = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()


# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(actual_closes, label='Actual')
plt.plot(predicted_closes, label='Predicted')
plt.legend()
plt.title('Actual vs Predicted Close Prices')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.show()

# Calculate and print the Mean Absolute Error
mae = np.mean(np.abs(predicted_closes - actual_closes))
print(f"Mean Absolute Error: ${mae:.2f}")